In [1]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
#from pytorch_tabnet.tab_model import TabNetModel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
import warnings
import shap
import optuna
import torch
from sklearn import svm
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tqdm.auto import tqdm
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from pycaret.regression import *
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import ExtraTreeRegressor
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostClassifier, Pool
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

In [2]:
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)

In [82]:
train_data = pd.read_csv("./train.csv")

In [83]:
train_data['YEAR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.year
train_data['MONTH'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.month
train_data['DAY'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day
train_data['HOUR'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.hour
train_data['MINUTE'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.minute
train_data['DAY_OF_WEEK'] =  pd.to_datetime(train_data['TIMESTAMP']).dt.day_name()

In [84]:
train_data = train_data.drop(['MONTH', 'DAY', 'YEAR', 'MINUTE'], axis = 1)

In [85]:
colls = []
for coll in train_data :
    if 'X' in coll and train_data[coll].isna().sum() == 598 :
        colls.append(coll)
print(len(colls))

82


In [86]:
train_data.drop(colls, inplace = True, axis = 1)

In [87]:
train_data.loc[train_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
train_data.loc[train_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
train_data.loc[train_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

train_data.loc[train_data.DAY_OF_WEEK == 'Monday','DAY_OF_WEEK'] = 1
train_data.loc[train_data.DAY_OF_WEEK == 'Tuesday','DAY_OF_WEEK'] = 2
train_data.loc[train_data.DAY_OF_WEEK == 'Wednesday','DAY_OF_WEEK'] = 3
train_data.loc[train_data.DAY_OF_WEEK == 'Thursday','DAY_OF_WEEK'] = 4
train_data.loc[train_data.DAY_OF_WEEK == 'Friday','DAY_OF_WEEK'] = 5
train_data.loc[train_data.DAY_OF_WEEK == 'Saturday','DAY_OF_WEEK'] = 6
train_data.loc[train_data.DAY_OF_WEEK == 'Sunday','DAY_OF_WEEK'] = 7

train_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
train_data

,Y_Class,Y_Quality,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK
0,1,0.533433,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN,5,1
1,2,0.541819,T050307,1,NaN,NaN,NaN,NaN,NaN,NaN,...,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN,5,1
2,1,0.531267,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN,5,1
3,2,0.537325,T050307,1,NaN,NaN,NaN,NaN,NaN,NaN,...,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,NaN,5,1
4,1,0.531590,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,NaN,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,1,0.526546,T100306,2,2.0,95.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,4
594,0,0.524022,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,369.811321,353.0,49.47,53.07,50.89,55.10,66.49,1.0,22,4
595,0,0.521289,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,...,367.018868,352.0,NaN,NaN,NaN,NaN,NaN,1.0,22,4
596,1,0.531375,T100304,3,40.0,94.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,4


In [88]:
T050304 = train_data[train_data['LINE'] == 'T050304']
T050307 = train_data[train_data['LINE'] == 'T050307']
T010305 = train_data[train_data['LINE'] == 'T010305']
T010306 = train_data[train_data['LINE'] == 'T010306']
T100304 = train_data[train_data['LINE'] == 'T100304']
T100306 = train_data[train_data['LINE'] == 'T100306']

In [89]:
T050304.drop(['LINE'], axis = 1, inplace = True)
T050307.drop(['LINE'], axis = 1, inplace = True)
T010305.drop(['LINE'], axis = 1, inplace = True)
T010306.drop(['LINE'], axis = 1, inplace = True)
T100304.drop(['LINE'], axis = 1, inplace = True)
T100306.drop(['LINE'], axis = 1, inplace = True)

In [90]:
T050304_col = []
for col in T050304 :
    if 'X' in col and T050304[col].isna().sum() == len(T050304) :
        T050304_col.append(col)
print(len(T050304_col))

T050304.drop(T050304_col, inplace = True, axis = 1)

825


In [91]:
T050307_col = []
for col in T050307 :
    if 'X' in col and T050307[col].isna().sum() == len(T050307) :
        T050307_col.append(col)
print(len(T050307_col))

T050307.drop(T050307_col, inplace = True, axis = 1)

818


In [92]:
T010305_col = []
for col in T010305 :
    if 'X' in col and T010305[col].isna().sum() == len(T010305) :
        T010305_col.append(col)
print(len(T010305_col))

T010305.drop(T010305_col, inplace = True, axis = 1)

1907


In [93]:
T010306_col = []
for col in T010306 :
    if 'X' in col and T010306[col].isna().sum() == len(T010306) :
        T010306_col.append(col)
print(len(T010306_col))

T010306.drop(T010306_col, inplace = True, axis = 1)

1907


In [94]:
T100304_col = []
for col in T100304 :
    if 'X' in col and T100304[col].isna().sum() == len(T100304) :
        T100304_col.append(col)
print(len(T100304_col))

T100304.drop(T100304_col, inplace = True, axis = 1)

2122


In [95]:
T100306_col = []
for col in T100306 :
    if 'X' in col and T100306[col].isna().sum() == len(T100306) :
        T100306_col.append(col)
print(len(T100306_col))

T100306.drop(T100306_col, inplace = True, axis = 1)

2122


In [96]:
T051 = list(set(T050307_col)-set(T050304_col))
T052 = list(set(T050304_col)-set(T050307_col))
T5drop = T051 + T052

T011 = list(set(T010305_col)-set(T010306_col))
T012 = list(set(T010306_col)-set(T010305_col))
T0drop = T011 + T012

T101 = list(set(T100304_col)-set(T100306_col))
T102 = list(set(T100306_col)-set(T100304_col))
T1drop = T101 + T102

In [97]:
T05 = pd.concat([T050304, T050307])
T01 = pd.concat([T010305, T010306])
T10 = pd.concat([T100304, T100306])

In [98]:
T05.drop(T5drop, inplace = True, axis = 1)
T01.drop(T0drop, inplace = True, axis = 1)
T10.drop(T1drop, inplace = True, axis = 1)

In [99]:
T05 = T05.fillna(0)
T01 = T01.fillna(0)
T10 = T10.fillna(0)

# T05

In [100]:
T05_class = T05['Y_Class']
T05_target = T05['Y_Quality']
T05_data = T05.drop(['Y_Class', 'Y_Quality'], axis = 1)

In [101]:
T_05c = T05_data.columns
T_05scaler = RobustScaler()
T_05scaled = T_05scaler.fit_transform(T05_data)
T05_data = pd.DataFrame(T_05scaled, columns = T_05c)
T05_data

,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK
0,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.024793,-0.052154,...,0.058361,1.0,-0.819981,-0.721628,-1.242857,-0.958879,3.252475,-1.0,-0.222222,-0.6
1,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,0.024793,-0.029478,...,-0.511376,1.0,-0.829261,-0.942865,-0.535000,-0.848622,3.396040,-1.0,-0.222222,-0.6
2,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,-0.493215,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054455,-1.0,-0.222222,-0.6
3,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,0.002755,-0.043084,...,0.134068,1.0,-1.243118,-0.693973,-0.828571,-1.106191,3.044554,-1.0,-0.111111,-0.6
4,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.002755,-0.047619,...,3.097439,1.0,-0.639963,-0.602661,0.402857,-0.996385,3.433168,-1.0,-0.111111,-0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.0,-1.0,-1.0,-1.0,0.0,-0.497972,-0.500000,-0.502292,-1.049587,-0.882086,...,-0.709724,-8.0,0.023198,0.080355,0.227143,0.044736,0.346535,0.0,0.111111,0.2
116,0.0,-1.0,-1.0,-1.0,0.0,-0.495943,-0.497972,-0.500255,-1.049587,-0.891156,...,-3.013162,-10.0,0.000928,-0.084529,0.524286,0.057840,0.544554,0.0,0.111111,0.2
117,0.0,-1.0,-1.0,-1.0,0.0,-0.491886,-0.493915,-0.496179,-1.044077,-0.886621,...,0.722783,0.0,-0.019487,0.079311,0.322143,-0.214641,-0.519802,0.0,0.222222,0.2
118,0.0,-1.0,-1.0,-1.0,0.0,-0.491886,-0.493915,-0.496179,-1.033058,-0.877551,...,0.700745,0.0,0.064646,0.218106,-0.007857,0.106191,-0.368812,0.0,0.777778,0.2


In [102]:
T05_y_class = pd.DataFrame(T05_target, columns = ['Y_Quality'])
T05_idx = T05_y_class.index
T05_y_class = T05_y_class.reset_index()
T05_data = pd.concat([T05_data, T05_y_class], axis = 1)
T05_data = T05_data.set_index('index')
T05_data.drop(['Y_Quality'], axis = 1, inplace = True)
T05_data

,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.024793,-0.052154,...,0.058361,1.0,-0.819981,-0.721628,-1.242857,-0.958879,3.252475,-1.0,-0.222222,-0.6
2,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,0.024793,-0.029478,...,-0.511376,1.0,-0.829261,-0.942865,-0.535000,-0.848622,3.396040,-1.0,-0.222222,-0.6
4,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,-0.493215,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054455,-1.0,-0.222222,-0.6
6,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,0.002755,-0.043084,...,0.134068,1.0,-1.243118,-0.693973,-0.828571,-1.106191,3.044554,-1.0,-0.111111,-0.6
8,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.002755,-0.047619,...,3.097439,1.0,-0.639963,-0.602661,0.402857,-0.996385,3.433168,-1.0,-0.111111,-0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,0.0,-1.0,-1.0,-1.0,0.0,-0.497972,-0.500000,-0.502292,-1.049587,-0.882086,...,-0.709724,-8.0,0.023198,0.080355,0.227143,0.044736,0.346535,0.0,0.111111,0.2
550,0.0,-1.0,-1.0,-1.0,0.0,-0.495943,-0.497972,-0.500255,-1.049587,-0.891156,...,-3.013162,-10.0,0.000928,-0.084529,0.524286,0.057840,0.544554,0.0,0.111111,0.2
551,0.0,-1.0,-1.0,-1.0,0.0,-0.491886,-0.493915,-0.496179,-1.044077,-0.886621,...,0.722783,0.0,-0.019487,0.079311,0.322143,-0.214641,-0.519802,0.0,0.222222,0.2


In [103]:
#13
setup_rgs = setup(data = T05_data, target = T05_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,4
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(120, 1960)"
4,Transformed data shape,"(120, 1960)"
5,Transformed train set shape,"(84, 1960)"
6,Transformed test set shape,"(36, 1960)"
7,Numeric features,1959
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0046,0.0000,0.0061,0.5948,0.0040,0.0086,0.2520
catboost,CatBoost Regressor,0.0050,0.0001,0.0063,0.6362,0.0041,0.0092,12.8730
knn,K Neighbors Regressor,0.0079,0.0001,0.0107,-0.1817,0.0069,0.0147,0.0420
lasso,Lasso Regression,0.0083,0.0001,0.0105,-0.1075,0.0069,0.0154,0.2290
en,Elastic Net,0.0082,0.0001,0.0106,-0.1176,0.0069,0.0153,0.0420
llar,Lasso Least Angle Regression,0.0082,0.0001,0.0105,-0.1016,0.0068,0.0154,0.0410
xgboost,Extreme Gradient Boosting,0.0053,0.0001,0.0069,0.4899,0.0045,0.0098,0.1820
dt,Decision Tree Regressor,0.0081,0.0001,0.0109,-0.3774,0.0071,0.0150,0.0430
rf,Random Forest Regressor,0.0054,0.0001,0.0069,0.5107,0.0045,0.0101,0.2960
ada,AdaBoost Regressor,0.0051,0.0001,0.0065,0.5521,0.0042,0.0096,0.1360


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [104]:
T05_total_models = []

T05_model_lightgbm = create_model('lightgbm', fold = 5)
T05_total_models.append(T05_model_lightgbm)

T05_model_rf = create_model('rf', fold = 5)
T05_total_models.append(T05_model_rf)

T05_model_gbr = create_model('gbr', fold = 5)
T05_total_models.append(T05_model_gbr)

T05_model_et = create_model('et', fold = 5)
T05_total_models.append(T05_model_et)

T05_model_catboost = create_model('catboost', fold = 5)
T05_total_models.append(T05_model_catboost)

T05_model_xgb = create_model('xgboost', fold = 5)
T05_total_models.append(T05_model_xgb)

T05_model_ada = create_model('ada', fold = 5)

T05_total_models.append(T05_model_ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0087,0.0002,0.0124,0.4187,0.0080,0.0161
1,0.0043,0.0000,0.0064,0.5686,0.0042,0.0080
2,0.0044,0.0000,0.0054,0.6445,0.0035,0.0082
3,0.0066,0.0001,0.0090,0.1046,0.0059,0.0124
4,0.0042,0.0000,0.0055,0.6568,0.0036,0.0079
Mean,0.0056,0.0001,0.0077,0.4786,0.0050,0.0105
Std,0.0018,0.0000,0.0027,0.2053,0.0017,0.0033


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0091,0.0001,0.0122,0.4350,0.0079,0.0169
1,0.0042,0.0000,0.0050,0.7384,0.0033,0.0078
2,0.0039,0.0000,0.0046,0.7433,0.0030,0.0074
3,0.0054,0.0001,0.0076,0.3583,0.0050,0.0101
4,0.0053,0.0001,0.0074,0.3691,0.0048,0.0100
Mean,0.0056,0.0001,0.0074,0.5288,0.0048,0.0104
Std,0.0019,0.0000,0.0027,0.1751,0.0017,0.0034


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0080,0.0001,0.0111,0.5341,0.0072,0.0148
1,0.0040,0.0000,0.0050,0.7386,0.0033,0.0074
2,0.0038,0.0000,0.0044,0.7663,0.0029,0.0072
3,0.0055,0.0001,0.0072,0.4291,0.0047,0.0104
4,0.0061,0.0001,0.0086,0.1467,0.0056,0.0114
Mean,0.0055,0.0001,0.0073,0.5229,0.0047,0.0103
Std,0.0015,0.0000,0.0024,0.2263,0.0016,0.0028


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0075,0.0001,0.0106,0.5714,0.0069,0.0139
1,0.0033,0.0000,0.0042,0.8116,0.0028,0.0061
2,0.0052,0.0000,0.0069,0.4260,0.0045,0.0100
3,0.0049,0.0000,0.0068,0.4879,0.0044,0.0093
4,0.0046,0.0001,0.0072,0.4067,0.0046,0.0086
Mean,0.0051,0.0001,0.0071,0.5407,0.0046,0.0096
Std,0.0014,0.0000,0.0020,0.1472,0.0013,0.0025


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0078,0.0001,0.0108,0.5597,0.0070,0.0144
1,0.0040,0.0000,0.0046,0.7818,0.0030,0.0074
2,0.0043,0.0000,0.0050,0.7003,0.0033,0.0081
3,0.0060,0.0001,0.0079,0.3076,0.0052,0.0113
4,0.0052,0.0000,0.0068,0.4613,0.0045,0.0098
Mean,0.0055,0.0001,0.0070,0.5621,0.0046,0.0102
Std,0.0014,0.0000,0.0022,0.1687,0.0014,0.0025


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [105]:
T05_model_lightgbm = tune_model(T05_model_lightgbm, fold=5, choose_better = True)

T05_model_rf = tune_model(T05_model_rf, fold=5, choose_better = True)

T05_model_gbr = tune_model(T05_model_gbr, fold=5, choose_better = True)

T05_model_et = tune_model(T05_model_et, fold=5, choose_better = True)

T05_model_catboost = tune_model(T05_model_catboost, fold=5, choose_better = True)

T05_model_xgb = tune_model(T05_model_xgb, fold=5, choose_better = True)

T05_model_ada = tune_model(T05_model_ada, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0119,0.0003,0.0163,-0.0088,0.0106,0.0220
1,0.0068,0.0001,0.0100,-0.0538,0.0065,0.0127
2,0.0073,0.0001,0.0092,-0.0336,0.0060,0.0139
3,0.0077,0.0001,0.0096,-0.0169,0.0062,0.0144
4,0.0077,0.0001,0.0094,-0.0100,0.0061,0.0145
Mean,0.0083,0.0001,0.0109,-0.0247,0.0071,0.0155
Std,0.0018,0.0001,0.0027,0.0171,0.0017,0.0033


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0107,0.0002,0.0146,0.1955,0.0094,0.0197
1,0.0043,0.0001,0.0073,0.4400,0.0047,0.0080
2,0.0056,0.0001,0.0071,0.3869,0.0046,0.0106
3,0.0056,0.0001,0.0081,0.2680,0.0053,0.0106
4,0.0041,0.0000,0.0055,0.6521,0.0036,0.0076
Mean,0.0061,0.0001,0.0085,0.3885,0.0055,0.0113
Std,0.0024,0.0001,0.0031,0.1574,0.0020,0.0044


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0080,0.0001,0.0109,0.5498,0.0070,0.0148
1,0.0042,0.0000,0.0051,0.7270,0.0033,0.0079
2,0.0042,0.0000,0.0047,0.7339,0.0031,0.0079
3,0.0051,0.0001,0.0073,0.4087,0.0048,0.0096
4,0.0057,0.0001,0.0079,0.2904,0.0051,0.0108
Mean,0.0055,0.0001,0.0072,0.5420,0.0047,0.0102
Std,0.0014,0.0000,0.0022,0.1745,0.0014,0.0026


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [106]:
T05_tuned_models = [T05_model_lightgbm, T05_model_rf, T05_model_gbr, T05_model_et, T05_model_catboost, T05_model_xgb, T05_model_ada]
T05_tuned_models

[LGBMRegressor(random_state=4),
 RandomForestRegressor(n_jobs=-1, random_state=4),
 GradientBoostingRegressor(learning_rate=0.15, max_depth=1, max_features='log2',
                           min_impurity_decrease=0.0001, min_samples_leaf=3,
                           min_samples_split=5, n_estimators=190, random_state=4,
                           subsample=0.65),
 ExtraTreesRegressor(n_jobs=-1, random_state=4),
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
          

In [107]:
for model in T05_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0057,0.0001,0.0075,0.3715,0.0049,0.0107


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,1.111111,0.4,0.526346,0.534627
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.222222,-0.6,0.531590,0.538058
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,1.777778,-0.2,0.527398,0.529219
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,-0.666667,0.2,0.537733,0.534449
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,1.333333,-0.2,0.516711,0.525321
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,0.000000,-0.2,0.528027,0.526165
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.111111,-0.6,0.531503,0.535638
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.111111,-0.4,0.535405,0.528538
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.111111,-0.6,0.538049,0.537446
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,1.333333,-0.2,0.519451,0.524440


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0055,0.0000,0.0069,0.4650,0.0045,0.0103


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,1.111111,0.4,0.526346,0.532339
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.222222,-0.6,0.531590,0.532074
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,1.777778,-0.2,0.527398,0.526473
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,-0.666667,0.2,0.537733,0.533030
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,1.333333,-0.2,0.516711,0.526123
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,0.000000,-0.2,0.528027,0.526494
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.111111,-0.6,0.531503,0.532415
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.111111,-0.4,0.535405,0.526259
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.111111,-0.6,0.538049,0.537086
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,1.333333,-0.2,0.519451,0.527012


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0046,0.0000,0.0060,0.5954,0.0039,0.0086


...

,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,1.111111,0.4,0.526346,0.532170
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.222222,-0.6,0.531590,0.533534
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,1.777778,-0.2,0.527398,0.527295
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,-0.666667,0.2,0.537733,0.533111
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,1.333333,-0.2,0.516711,0.525592
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,0.000000,-0.2,0.528027,0.526128
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.111111,-0.6,0.531503,0.534414
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.111111,-0.4,0.535405,0.525635
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.111111,-0.6,0.538049,0.537445
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,1.333333,-0.2,0.519451,0.524647


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.0049,0.0000,0.0064,0.5452,0.0042,0.0093


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,1.111111,0.4,0.526346,0.532221
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.222222,-0.6,0.531590,0.536156
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,1.777778,-0.2,0.527398,0.529482
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,-0.666667,0.2,0.537733,0.532245
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,1.333333,-0.2,0.516711,0.527424
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,0.000000,-0.2,0.528027,0.527452
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.111111,-0.6,0.531503,0.532728
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.111111,-0.4,0.535405,0.526450
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.111111,-0.6,0.538049,0.535470
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,1.333333,-0.2,0.519451,0.524594


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0053,0.0000,0.0068,0.4788,0.0045,0.0100


,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
84,0.0,0.0,0.0,0.0,0.0,0.406694,0.404665,0.400408,0.702479,0.637188,...,0.318899,0.561440,0.165000,0.055581,-0.069307,0.0,1.111111,0.4,0.526346,0.531313
85,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.222222,-0.6,0.531590,0.531823
86,0.0,0.0,0.0,-1.0,0.0,0.266734,0.264706,0.265920,-0.112948,-0.083900,...,0.133931,0.010957,0.694286,0.066426,0.012376,0.0,1.777778,-0.2,0.527398,0.527378
87,0.0,0.0,0.0,-1.0,0.0,0.662272,0.660243,0.657157,1.325069,1.190476,...,0.203835,0.453953,-0.517857,0.077723,0.388614,0.0,-0.666667,0.2,0.537733,0.532867
88,0.0,0.0,0.0,0.0,0.0,-0.412779,-0.414807,-0.416709,-0.663912,-0.632653,...,0.263842,-0.222280,-0.676429,0.091731,0.430693,0.0,1.333333,-0.2,0.516711,0.526791
89,0.0,0.0,0.0,0.0,0.0,0.901623,0.899594,0.903719,0.741047,0.691610,...,-0.118466,0.130446,0.222143,0.048351,0.606436,0.0,0.000000,-0.2,0.528027,0.526925
90,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.052154,...,-1.266625,-1.042525,0.355000,-1.000904,2.888614,-1.0,-0.111111,-0.6,0.531503,0.533221
91,0.0,0.0,0.0,-1.0,0.0,0.191684,0.189655,0.190525,-0.162534,-0.147392,...,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,-0.111111,-0.4,0.535405,0.527342
92,0.0,0.0,0.0,0.0,0.0,0.177485,0.175456,0.176261,0.338843,0.342404,...,-1.125580,-0.983042,0.028571,-0.761862,1.680693,-1.0,-0.111111,-0.6,0.538049,0.535046
93,0.0,0.0,0.0,-1.0,0.0,-0.412779,-0.414807,-0.416709,-0.713499,-0.664399,...,-0.266935,-0.181059,0.390714,0.074108,0.695545,0.0,1.333333,-0.2,0.519451,0.526791


In [108]:
T05_final_regression_model = finalize_model(T05_model_et)

In [109]:
T05_prediction = predict_model(T05_final_regression_model, data=T05_data)
T05_prediction

,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK,prediction_label
0,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.024793,-0.052154,...,1.0,-0.819981,-0.721628,-1.242857,-0.958879,3.252475,-1.0,-0.222222,-0.6,0.533433
1,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,0.024793,-0.029478,...,1.0,-0.829261,-0.942865,-0.535000,-0.848622,3.396039,-1.0,-0.222222,-0.6,0.531267
2,0.0,-1.0,-1.0,-2.0,0.0,-0.199797,-0.201826,-0.208864,-0.008264,-0.043084,...,0.0,-0.859573,-0.669450,-0.216429,-1.004067,3.054456,-1.0,-0.222222,-0.6,0.531590
3,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,0.002755,-0.043084,...,1.0,-1.243118,-0.693973,-0.828571,-1.106191,3.044554,-1.0,-0.111111,-0.6,0.533665
4,0.0,-1.0,-1.0,-1.0,0.0,-0.199797,-0.201826,-0.208864,-0.002755,-0.047619,...,1.0,-0.639963,-0.602661,0.402857,-0.996385,3.433168,-1.0,-0.111111,-0.6,0.531821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.0,-1.0,-1.0,-1.0,0.0,-0.497972,-0.500000,-0.502292,-1.049587,-0.882086,...,-8.0,0.023198,0.080355,0.227143,0.044736,0.346535,0.0,0.111111,0.2,0.533711
116,0.0,-1.0,-1.0,-1.0,0.0,-0.495943,-0.497972,-0.500255,-1.049587,-0.891156,...,-10.0,0.000928,-0.084529,0.524286,0.057840,0.544554,0.0,0.111111,0.2,0.533695
117,0.0,-1.0,-1.0,-1.0,0.0,-0.491886,-0.493915,-0.496179,-1.044077,-0.886621,...,0.0,-0.019487,0.079311,0.322143,-0.214641,-0.519802,0.0,0.222222,0.2,0.525271
118,0.0,-1.0,-1.0,-1.0,0.0,-0.491886,-0.493915,-0.496179,-1.033058,-0.877551,...,0.0,0.064646,0.218106,-0.007857,0.106191,-0.368812,0.0,0.777778,0.2,0.529806


In [110]:
T05_datas = T05_prediction['prediction_label']
T05_datas = pd.DataFrame(T05_datas.values, columns = ['Y_Quality'])
T05_datas

,Y_Quality
0,0.533433
1,0.531267
2,0.531590
3,0.533665
4,0.531821
...,...
115,0.533711
116,0.533695
117,0.525271
118,0.529806


In [111]:
T05_xtrain, T05_xtest, T05_ytrain, T05_ytest = train_test_split(T05_datas, T05_class, test_size = 0.1, random_state = 4)

In [112]:
scores = []
for thisC in [*range(1,100)]:
    svc=SVC(kernel='rbf',C=thisC, decision_function_shape = 'ovo')
    model=svc.fit(T05_xtrain,T05_ytrain)
    scoreTrain=model.score(T05_xtrain,T05_ytrain)
    scoreTest=model.score(T05_xtest,T05_ytest)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.972222, test score:0.916667
선형 SVM : C:2, training score:0.962963, test score:0.916667
선형 SVM : C:3, training score:0.981481, test score:0.916667
선형 SVM : C:4, training score:0.981481, test score:0.916667
선형 SVM : C:5, training score:0.972222, test score:0.916667
선형 SVM : C:6, training score:0.972222, test score:0.916667
선형 SVM : C:7, training score:0.972222, test score:0.916667
선형 SVM : C:8, training score:0.972222, test score:0.916667
선형 SVM : C:9, training score:0.972222, test score:0.916667
선형 SVM : C:10, training score:0.972222, test score:0.916667
선형 SVM : C:11, training score:0.972222, test score:0.916667
선형 SVM : C:12, training score:0.972222, test score:0.916667
선형 SVM : C:13, training score:0.972222, test score:0.916667
선형 SVM : C:14, training score:0.972222, test score:0.916667
선형 SVM : C:15, training score:0.972222, test score:0.916667
선형 SVM : C:16, training score:0.972222, test score:0.916667
선형 SVM : C:17, training score:0.972222, test scor

In [113]:
T05_svm = svm.SVC(kernel = 'rbf', decision_function_shape = 'ovo', C = 41)
T05_svm.fit(T05_xtrain, T05_ytrain)
T05_svm.predict(T05_xtest)

array([1, 1, 1, 2, 0, 1, 1, 2, 2, 0, 2, 2], dtype=int64)

# T01

In [114]:
T01_class = T01['Y_Class']
T01_target = T01['Y_Quality']
T01_data = T01.drop(['Y_Class', 'Y_Quality'], axis = 1)
T01_data

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK
28,1,1.0,89.0,0.0,45.0,11.0,90.0,45.0,11.0,42.0,...,182.758621,175.7,182.0,174.727586,160.5,446.0,429.716981,413.0,20,7
33,1,1.0,95.0,0.0,45.0,11.0,98.0,45.0,11.0,42.0,...,184.365517,171.4,175.7,168.258621,158.3,446.0,429.301887,413.0,7,2
39,1,1.0,95.0,0.0,45.0,11.0,102.0,45.0,12.0,42.0,...,173.068965,163.7,184.1,173.479310,165.8,447.0,427.867925,407.0,4,3
40,1,1.0,95.0,0.0,45.0,11.0,105.0,45.0,11.0,42.0,...,180.396429,152.7,191.8,178.832143,168.6,446.0,428.403846,413.0,12,3
41,1,1.0,87.0,0.0,45.0,11.0,105.0,45.0,11.0,42.0,...,172.751724,146.9,182.4,170.379310,162.9,446.0,429.094340,406.0,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,1,1.0,93.0,107.0,45.0,11.0,0.0,45.0,12.0,51.0,...,193.278571,177.5,183.3,169.553571,154.2,456.0,438.132075,413.0,15,2
501,1,2.0,95.0,124.0,45.0,11.0,0.0,45.0,12.0,51.0,...,197.392857,173.1,187.6,182.142857,175.1,466.0,445.307692,425.0,15,5
523,1,1.0,89.0,136.0,45.0,12.0,0.0,45.0,12.0,51.0,...,194.860714,183.9,189.2,181.700000,172.7,466.0,444.615385,423.0,10,7
531,1,1.0,90.0,142.0,45.0,11.0,0.0,45.0,12.0,51.0,...,190.407143,181.1,192.6,185.207143,173.2,466.0,446.038461,427.0,15,1


In [115]:
T_01c = T01_data.columns
T_01scaler = RobustScaler()
T_01scaled = T_01scaler.fit_transform(T01_data)
T01_data = pd.DataFrame(T_01scaled, columns = T_01c)
T01_data

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK
0,0.0,0.0,-0.25,-0.054348,0.0,0.0,1.011236,0.0,-1.0,-1.0,...,-0.390876,0.000000,-0.053030,0.321839,0.041096,-1.9,-1.452964,-5.0,0.777778,0.666667
1,0.0,0.0,1.25,-0.054348,0.0,0.0,1.101124,0.0,-1.0,-1.0,...,-0.287500,-0.222798,-0.530303,-0.217241,-0.109589,-1.9,-1.500816,-5.0,-0.666667,-1.000000
2,0.0,0.0,1.25,-0.054348,0.0,0.0,1.146067,0.0,0.0,-1.0,...,-1.014237,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-1.000000,-0.666667
3,0.0,0.0,1.25,-0.054348,0.0,0.0,1.179775,0.0,-1.0,-1.0,...,-0.542842,-1.191710,0.689394,0.663885,0.595890,-1.9,-1.604342,-5.0,-0.111111,-0.666667
4,0.0,0.0,-0.75,-0.054348,0.0,0.0,1.179775,0.0,-1.0,-1.0,...,-1.034646,-1.492228,-0.022727,-0.040517,0.205479,-1.9,-1.524742,-8.5,-0.111111,-0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.0,0.0,0.75,1.108696,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.285900,0.093264,0.045455,-0.109329,-0.390411,-0.9,-0.482871,-5.0,0.222222,-1.000000
125,0.0,1.0,1.25,1.293478,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.550583,-0.134715,0.371212,0.939778,1.041096,0.1,0.344334,1.0,0.222222,0.000000
126,0.0,0.0,-0.25,1.423913,0.0,1.0,0.000000,0.0,0.0,0.0,...,0.387683,0.424870,0.492424,0.902874,0.876712,0.1,0.264525,0.0,-0.333333,0.666667
127,0.0,0.0,0.00,1.489130,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.101173,0.279793,0.750000,1.195135,0.910959,0.1,0.428577,2.0,0.222222,-1.333333


In [116]:
T01_y_class = pd.DataFrame(T01_target, columns = ['Y_Quality'])
T01_idx = T01_y_class.index
T01_y_class = T01_y_class.reset_index()
T01_data = pd.concat([T01_data, T01_y_class], axis = 1)
T01_data = T01_data.set_index('index')
T01_data.drop(['Y_Quality'], axis = 1, inplace = True)
T01_data

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK
index,,,,,,,,,,,,,,,,,,,,,
28,0.0,0.0,-0.25,-0.054348,0.0,0.0,1.011236,0.0,-1.0,-1.0,...,-0.390876,0.000000,-0.053030,0.321839,0.041096,-1.9,-1.452964,-5.0,0.777778,0.666667
33,0.0,0.0,1.25,-0.054348,0.0,0.0,1.101124,0.0,-1.0,-1.0,...,-0.287500,-0.222798,-0.530303,-0.217241,-0.109589,-1.9,-1.500816,-5.0,-0.666667,-1.000000
39,0.0,0.0,1.25,-0.054348,0.0,0.0,1.146067,0.0,0.0,-1.0,...,-1.014237,-0.621762,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-1.000000,-0.666667
40,0.0,0.0,1.25,-0.054348,0.0,0.0,1.179775,0.0,-1.0,-1.0,...,-0.542842,-1.191710,0.689394,0.663885,0.595890,-1.9,-1.604342,-5.0,-0.111111,-0.666667
41,0.0,0.0,-0.75,-0.054348,0.0,0.0,1.179775,0.0,-1.0,-1.0,...,-1.034646,-1.492228,-0.022727,-0.040517,0.205479,-1.9,-1.524742,-8.5,-0.111111,-0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,0.0,0.0,0.75,1.108696,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.285900,0.093264,0.045455,-0.109329,-0.390411,-0.9,-0.482871,-5.0,0.222222,-1.000000
501,0.0,1.0,1.25,1.293478,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.550583,-0.134715,0.371212,0.939778,1.041096,0.1,0.344334,1.0,0.222222,0.000000
523,0.0,0.0,-0.25,1.423913,0.0,1.0,0.000000,0.0,0.0,0.0,...,0.387683,0.424870,0.492424,0.902874,0.876712,0.1,0.264525,0.0,-0.333333,0.666667


In [117]:
from pycaret.regression import *
setup_rgs = setup(data = T01_data, target = T01_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,4
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(129, 882)"
4,Transformed data shape,"(129, 882)"
5,Transformed train set shape,"(90, 882)"
6,Transformed test set shape,"(39, 882)"
7,Numeric features,881
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,0.0048,0.0000,0.0062,0.2522,0.0041,0.0091,0.0330
et,Extra Trees Regressor,0.0039,0.0000,0.0048,0.6499,0.0031,0.0074,0.1550
rf,Random Forest Regressor,0.0043,0.0000,0.0053,0.5720,0.0034,0.0081,0.1910
catboost,CatBoost Regressor,0.0045,0.0000,0.0058,0.5143,0.0038,0.0085,6.8180
ada,AdaBoost Regressor,0.0043,0.0000,0.0053,0.5531,0.0035,0.0081,0.0930
gbr,Gradient Boosting Regressor,0.0043,0.0000,0.0054,0.5344,0.0035,0.0081,0.1360
xgboost,Extreme Gradient Boosting,0.0048,0.0000,0.0060,0.4576,0.0040,0.0091,0.0990
omp,Orthogonal Matching Pursuit,0.0066,0.0001,0.0079,-0.2229,0.0052,0.0124,0.0310
lightgbm,Light Gradient Boosting Machine,0.0055,0.0001,0.0073,0.2391,0.0047,0.0103,0.0570
llar,Lasso Least Angle Regression,0.0069,0.0001,0.0093,-0.0920,0.0061,0.0129,0.0310


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [118]:
T01_total_models = []

T01_model_lightgbm = create_model('lightgbm', fold = 5)
T01_total_models.append(T01_model_lightgbm)

T01_model_rf = create_model('rf', fold = 5)
T01_total_models.append(T01_model_rf)

T01_model_gbr = create_model('gbr', fold = 5)
T01_total_models.append(T01_model_gbr)

T01_model_et = create_model('et', fold = 5)
T01_total_models.append(T01_model_et)

T01_model_catboost = create_model('catboost', fold = 5)
T01_total_models.append(T01_model_catboost)

T01_model_xgb = create_model('xgboost', fold = 5)
T01_total_models.append(T01_model_xgb)

T01_model_ada = create_model('ada', fold = 5)
T01_total_models.append(T01_model_ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0055,0.0000,0.0067,0.1802,0.0044,0.0104
1,0.0056,0.0001,0.0078,0.3765,0.0051,0.0104
2,0.0041,0.0000,0.0065,-0.2852,0.0043,0.0078
3,0.0066,0.0001,0.0096,0.4852,0.0062,0.0125
4,0.0064,0.0001,0.0087,0.1887,0.0057,0.0118
Mean,0.0056,0.0001,0.0078,0.1891,0.0051,0.0106
Std,0.0009,0.0000,0.0012,0.2638,0.0008,0.0016


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0041,0.0000,0.0053,0.4971,0.0034,0.0077
1,0.0048,0.0000,0.0060,0.6271,0.0039,0.0091
2,0.0043,0.0000,0.0052,0.1795,0.0034,0.0081
3,0.0060,0.0001,0.0077,0.6650,0.0051,0.0113
4,0.0040,0.0000,0.0046,0.7727,0.0030,0.0075
Mean,0.0047,0.0000,0.0058,0.5483,0.0038,0.0087
Std,0.0007,0.0000,0.0011,0.2043,0.0007,0.0014


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0035,0.0000,0.0044,0.6474,0.0029,0.0066
1,0.0048,0.0000,0.0062,0.6025,0.0040,0.0090
2,0.0037,0.0000,0.0054,0.1218,0.0035,0.0070
3,0.0053,0.0001,0.0072,0.7069,0.0047,0.0100
4,0.0042,0.0000,0.0050,0.7269,0.0033,0.0078
Mean,0.0043,0.0000,0.0056,0.5611,0.0037,0.0081
Std,0.0007,0.0000,0.0010,0.2240,0.0006,0.0013


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0036,0.0000,0.0045,0.6279,0.0030,0.0067
1,0.0045,0.0000,0.0060,0.6325,0.0039,0.0085
2,0.0036,0.0000,0.0050,0.2312,0.0033,0.0069
3,0.0056,0.0001,0.0077,0.6644,0.0051,0.0107
4,0.0038,0.0000,0.0042,0.8116,0.0027,0.0070
Mean,0.0042,0.0000,0.0055,0.5935,0.0036,0.0079
Std,0.0008,0.0000,0.0013,0.1932,0.0008,0.0015


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0039,0.0000,0.0050,0.5505,0.0032,0.0074
1,0.0049,0.0000,0.0063,0.5944,0.0041,0.0093
2,0.0044,0.0000,0.0058,-0.0337,0.0038,0.0082
3,0.0058,0.0001,0.0074,0.6889,0.0049,0.0111
4,0.0060,0.0001,0.0078,0.3476,0.0051,0.0111
Mean,0.0050,0.0000,0.0065,0.4295,0.0042,0.0094
Std,0.0008,0.0000,0.0010,0.2571,0.0007,0.0015


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [119]:
T01_model_lightgbm = tune_model(T01_model_lightgbm, fold=5, choose_better = True)

T01_model_rf = tune_model(T01_model_rf, fold=5, choose_better = True)

T01_model_gbr = tune_model(T01_model_gbr, fold=5, choose_better = True)

T01_model_et = tune_model(T01_model_et, fold=5, choose_better = True)

T01_model_catboost = tune_model(T01_model_catboost, fold=5, choose_better = True)

T01_model_xgb = tune_model(T01_model_xgb, fold=5, choose_better = True)

T01_model_ada = tune_model(T01_model_ada, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0060,0.0001,0.0075,-0.0286,0.0049,0.0111
1,0.0068,0.0001,0.0098,-0.0033,0.0064,0.0129
2,0.0044,0.0000,0.0061,-0.1257,0.0040,0.0084
3,0.0098,0.0002,0.0134,-0.0089,0.0087,0.0185
4,0.0074,0.0001,0.0100,-0.0725,0.0065,0.0137
Mean,0.0069,0.0001,0.0094,-0.0478,0.0061,0.0129
Std,0.0018,0.0000,0.0025,0.0459,0.0016,0.0033


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0049,0.0000,0.0059,0.3761,0.0038,0.0091
1,0.0061,0.0001,0.0088,0.1986,0.0057,0.0114
2,0.0042,0.0000,0.0059,-0.0608,0.0039,0.0079
3,0.0077,0.0001,0.0110,0.3169,0.0072,0.0145
4,0.0063,0.0001,0.0086,0.2118,0.0056,0.0116
Mean,0.0058,0.0001,0.0080,0.2085,0.0052,0.0109
Std,0.0012,0.0000,0.0020,0.1500,0.0013,0.0022


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0038,0.0000,0.0050,0.5371,0.0033,0.0071
1,0.0049,0.0000,0.0063,0.5878,0.0041,0.0092
2,0.0038,0.0000,0.0047,0.3268,0.0031,0.0072
3,0.0066,0.0001,0.0082,0.6203,0.0054,0.0124
4,0.0045,0.0000,0.0050,0.7337,0.0032,0.0083
Mean,0.0047,0.0000,0.0059,0.5611,0.0038,0.0089
Std,0.0010,0.0000,0.0013,0.1338,0.0009,0.0019


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [120]:
T01_tuned_models = [T01_model_lightgbm, T01_model_rf, T01_model_gbr, T01_model_et, T01_model_catboost, T01_model_xgb, T01_model_ada]
T01_tuned_models

[LGBMRegressor(random_state=4),
 RandomForestRegressor(n_jobs=-1, random_state=4),
 GradientBoostingRegressor(random_state=4),
 ExtraTreesRegressor(n_jobs=-1, random_state=4),
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=4,
              reg_alpha=0, reg_lambda=1, ...),
 AdaBoostRegressor(learning_rate=0.0005, n_estimators=70, random_state=4)]

In [121]:
for model in T01_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0044,0.0000,0.0059,0.3650,0.0038,0.0082


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,0.0,1.247191,0.0,-1.0,-1.0,...,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,0.222222,-0.333333,0.533273,0.532169
91,0.0,0.0,-0.25,-0.054348,0.0,0.0,0.146067,0.0,0.0,-1.0,...,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.444444,0.333333,0.528856,0.522478
92,0.0,1.0,0.25,-0.054348,0.0,0.0,0.426966,0.0,-1.0,-1.0,...,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,-1.222222,0.333333,0.530497,0.527501
93,0.0,0.0,-0.25,1.597826,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-0.666667,-1.000000,0.548592,0.543613
94,0.0,0.0,-0.50,-0.054348,0.0,0.0,1.876405,0.0,-1.0,-1.0,...,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,-0.333333,0.666667,0.536433,0.536682
95,0.0,0.0,1.00,1.478261,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.000000,0.535394,0.532610
96,0.0,0.0,1.25,-0.054348,0.0,0.0,1.146067,0.0,0.0,-1.0,...,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-1.000000,-0.666667,0.531583,0.535107
97,0.0,0.0,-0.75,-0.054348,0.0,0.0,0.123596,0.0,-1.0,-1.0,...,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.888889,0.666667,0.526741,0.522504
98,0.0,0.0,1.00,1.413043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,0.222222,-0.333333,0.538357,0.532969
99,0.0,0.0,1.25,0.663043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,0.444444,-0.333333,0.531043,0.529707


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0035,0.0000,0.0047,0.6045,0.0030,0.0065


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,0.0,1.247191,0.0,-1.0,-1.0,...,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,0.222222,-0.333333,0.533273,0.530522
91,0.0,0.0,-0.25,-0.054348,0.0,0.0,0.146067,0.0,0.0,-1.0,...,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.444444,0.333333,0.528856,0.524496
92,0.0,1.0,0.25,-0.054348,0.0,0.0,0.426966,0.0,-1.0,-1.0,...,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,-1.222222,0.333333,0.530497,0.528356
93,0.0,0.0,-0.25,1.597826,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-0.666667,-1.000000,0.548592,0.541061
94,0.0,0.0,-0.50,-0.054348,0.0,0.0,1.876405,0.0,-1.0,-1.0,...,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,-0.333333,0.666667,0.536433,0.530182
95,0.0,0.0,1.00,1.478261,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.000000,0.535394,0.534484
96,0.0,0.0,1.25,-0.054348,0.0,0.0,1.146067,0.0,0.0,-1.0,...,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-1.000000,-0.666667,0.531583,0.530235
97,0.0,0.0,-0.75,-0.054348,0.0,0.0,0.123596,0.0,-1.0,-1.0,...,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.888889,0.666667,0.526741,0.529910
98,0.0,0.0,1.00,1.413043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,0.222222,-0.333333,0.538357,0.534410
99,0.0,0.0,1.25,0.663043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,0.444444,-0.333333,0.531043,0.528847


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0033,0.0000,0.0042,0.6838,0.0027,0.0062


...

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,0.0,1.247191,0.0,-1.0,-1.0,...,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,0.222222,-0.333333,0.533273,0.529830
91,0.0,0.0,-0.25,-0.054348,0.0,0.0,0.146067,0.0,0.0,-1.0,...,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.444444,0.333333,0.528856,0.523231
92,0.0,1.0,0.25,-0.054348,0.0,0.0,0.426966,0.0,-1.0,-1.0,...,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,-1.222222,0.333333,0.530497,0.527884
93,0.0,0.0,-0.25,1.597826,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-0.666667,-1.000000,0.548592,0.542182
94,0.0,0.0,-0.50,-0.054348,0.0,0.0,1.876405,0.0,-1.0,-1.0,...,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,-0.333333,0.666667,0.536433,0.533767
95,0.0,0.0,1.00,1.478261,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.000000,0.535394,0.538261
96,0.0,0.0,1.25,-0.054348,0.0,0.0,1.146067,0.0,0.0,-1.0,...,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-1.000000,-0.666667,0.531583,0.532756
97,0.0,0.0,-0.75,-0.054348,0.0,0.0,0.123596,0.0,-1.0,-1.0,...,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.888889,0.666667,0.526741,0.528219
98,0.0,0.0,1.00,1.413043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,0.222222,-0.333333,0.538357,0.535811
99,0.0,0.0,1.25,0.663043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,0.444444,-0.333333,0.531043,0.530089


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.0039,0.0000,0.0050,0.5431,0.0033,0.0073


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,0.0,1.247191,0.0,-1.0,-1.0,...,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,0.222222,-0.333333,0.533273,0.533141
91,0.0,0.0,-0.25,-0.054348,0.0,0.0,0.146067,0.0,0.0,-1.0,...,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.444444,0.333333,0.528856,0.530075
92,0.0,1.0,0.25,-0.054348,0.0,0.0,0.426966,0.0,-1.0,-1.0,...,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,-1.222222,0.333333,0.530497,0.529822
93,0.0,0.0,-0.25,1.597826,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-0.666667,-1.000000,0.548592,0.542850
94,0.0,0.0,-0.50,-0.054348,0.0,0.0,1.876405,0.0,-1.0,-1.0,...,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,-0.333333,0.666667,0.536433,0.532325
95,0.0,0.0,1.00,1.478261,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.000000,0.535394,0.533927
96,0.0,0.0,1.25,-0.054348,0.0,0.0,1.146067,0.0,0.0,-1.0,...,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-1.000000,-0.666667,0.531583,0.529020
97,0.0,0.0,-0.75,-0.054348,0.0,0.0,0.123596,0.0,-1.0,-1.0,...,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.888889,0.666667,0.526741,0.525841
98,0.0,0.0,1.00,1.413043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,0.222222,-0.333333,0.538357,0.529761
99,0.0,0.0,1.25,0.663043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,0.444444,-0.333333,0.531043,0.529630


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0041,0.0000,0.0054,0.4705,0.0035,0.0076


,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
90,0.0,0.0,0.25,-0.054348,0.0,0.0,1.247191,0.0,-1.0,-1.0,...,0.030303,0.324425,0.047945,-1.9,-1.650897,-5.0,0.222222,-0.333333,0.533273,0.529464
91,0.0,0.0,-0.25,-0.054348,0.0,0.0,0.146067,0.0,0.0,-1.0,...,-1.507576,-1.028376,-0.684932,0.1,0.792153,4.5,0.444444,0.333333,0.528856,0.523446
92,0.0,1.0,0.25,-0.054348,0.0,0.0,0.426966,0.0,-1.0,-1.0,...,0.159091,0.323409,-0.623288,-0.9,-0.317564,0.0,-1.222222,0.333333,0.530497,0.529347
93,0.0,0.0,-0.25,1.597826,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.204545,-0.174138,-0.397260,-1.9,-1.422512,-4.0,-0.666667,-1.000000,0.548592,0.537814
94,0.0,0.0,-0.50,-0.054348,0.0,0.0,1.876405,0.0,-1.0,-1.0,...,-2.007576,-1.426591,-0.616438,0.1,0.807671,4.5,-0.333333,0.666667,0.536433,0.530192
95,0.0,0.0,1.00,1.478261,0.0,1.0,0.000000,0.0,0.0,0.0,...,-0.333333,-0.419540,-0.500000,0.1,0.461120,-0.5,0.000000,0.000000,0.535394,0.535325
96,0.0,0.0,1.25,-0.054348,0.0,0.0,1.146067,0.0,0.0,-1.0,...,0.106061,0.217816,0.404110,-1.8,-1.666123,-8.0,-1.000000,-0.666667,0.531583,0.529464
97,0.0,0.0,-0.75,-0.054348,0.0,0.0,0.123596,0.0,-1.0,-1.0,...,-0.840909,-0.261710,0.356164,0.1,0.645836,5.0,0.888889,0.666667,0.526741,0.529462
98,0.0,0.0,1.00,1.413043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.916667,-0.801293,-1.465753,-0.8,-0.192161,0.0,0.222222,-0.333333,0.538357,0.535158
99,0.0,0.0,1.25,0.663043,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.340909,0.038985,-0.383562,-0.9,-0.132304,-0.5,0.444444,-0.333333,0.531043,0.529451


In [124]:
T01_final_regression_model = finalize_model(T01_model_gbr)

In [125]:
T01_prediction = predict_model(T01_final_regression_model, data=T01_data)

In [126]:
T01_datas = T01_prediction['prediction_label']
T01_datas = pd.DataFrame(T01_datas.values, columns = ['Y_Quality'])
T01_datas

,Y_Quality
0,0.521643
1,0.526851
2,0.531782
3,0.530605
4,0.528686
...,...
124,0.534831
125,0.533497
126,0.534386
127,0.559978


In [127]:
T01_xtrain, T01_xtest, T01_ytrain, T01_ytest = train_test_split(T01_datas, T01_class, test_size = 0.1, random_state = 4)

In [128]:
scores = []
for thisC in [*range(1,100)]:
    svc=SVC(kernel='rbf',C=thisC, decision_function_shape = 'ovo')
    model=svc.fit(T01_xtrain,T01_ytrain)
    scoreTrain=model.score(T01_xtrain,T01_ytrain)
    scoreTest=model.score(T01_xtest,T01_ytest)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.956897, test score:0.923077
선형 SVM : C:2, training score:0.991379, test score:1.000000
선형 SVM : C:3, training score:0.991379, test score:1.000000
선형 SVM : C:4, training score:0.982759, test score:1.000000
선형 SVM : C:5, training score:0.974138, test score:1.000000
선형 SVM : C:6, training score:0.991379, test score:1.000000
선형 SVM : C:7, training score:0.991379, test score:1.000000
선형 SVM : C:8, training score:0.991379, test score:1.000000
선형 SVM : C:9, training score:0.991379, test score:1.000000
선형 SVM : C:10, training score:0.991379, test score:1.000000
선형 SVM : C:11, training score:0.991379, test score:1.000000
선형 SVM : C:12, training score:0.991379, test score:1.000000
선형 SVM : C:13, training score:0.991379, test score:1.000000
선형 SVM : C:14, training score:0.974138, test score:1.000000
선형 SVM : C:15, training score:0.974138, test score:1.000000
선형 SVM : C:16, training score:0.974138, test score:1.000000
선형 SVM : C:17, training score:0.974138, test scor

In [129]:
T01_svm = svm.SVC(kernel = 'rbf', decision_function_shape = 'ovo', C = 2)
T01_svm.fit(T01_xtrain, T01_ytrain)
T01_svm.predict(T01_xtest)

array([1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 0, 1, 2], dtype=int64)

# T10

In [130]:
T10_class = T10['Y_Class']
T10_target = T10['Y_Quality']
T10_data = T10.drop(['Y_Class', 'Y_Quality'], axis = 1)
T10_data

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK
22,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,157.235484,150.9,326.0,309.290909,290.0,13.7,13.443333,13.2,8,2
23,2,2.0,102.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,9,2
25,2,2.0,97.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,154.074194,150.9,327.0,310.854546,298.0,13.7,13.454839,13.2,9,7
29,2,2.0,100.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,156.038710,149.1,327.0,312.109091,298.0,13.7,13.422581,13.2,23,7
37,2,2.0,100.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,...,155.829032,148.4,327.0,312.254546,298.0,13.7,13.456667,13.3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,2,1.0,94.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,154.309677,149.1,326.0,311.272727,298.0,13.7,13.483871,13.3,18,2
590,2,1.0,89.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,154.812903,150.0,327.0,313.400000,299.0,13.6,13.393548,13.2,1,3
591,2,1.0,87.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,154.777419,147.7,326.0,313.472727,298.0,13.6,13.400000,13.2,1,3
593,2,2.0,95.0,0.0,45.0,10.0,0.0,50.0,10.0,52.0,...,155.438710,149.0,327.0,313.363636,298.0,13.6,13.351613,13.2,14,4


In [131]:
T_10c = T10_data.columns
T_10scaler = RobustScaler()
T_10scaled = T_10scaler.fit_transform(T10_data)
T10_data = pd.DataFrame(T_10scaled, columns = T_10c)
T10_data

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK
0,0.0,0.0,1.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.962572,0.66,0.0,0.386364,0.111111,1.0,0.584058,0.0,-0.166667,-1.000000
1,0.0,0.0,1.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-45.815738,-29.52,-32.6,-38.275000,-32.111111,-136.0,-180.608692,-13.2,-0.083333,-1.000000
2,0.0,0.0,0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.022073,0.66,0.1,0.581818,1.000000,1.0,0.739130,0.0,-0.083333,0.666667
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.606526,0.30,0.1,0.738636,1.000000,1.0,0.304348,0.0,1.083333,0.666667
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.544146,0.16,0.1,0.756818,1.000000,1.0,0.763768,0.1,-0.583333,-0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,-1.0,-0.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.092131,0.30,0.0,0.634091,1.000000,1.0,1.130435,0.1,0.666667,-1.000000
345,0.0,-1.0,-1.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.241843,0.48,0.1,0.900000,1.111111,0.0,-0.086956,0.0,-0.750000,-0.666667
346,0.0,-1.0,-1.6,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.231286,0.02,0.0,0.909091,1.000000,0.0,0.000000,0.0,-0.750000,-0.666667
347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.428023,0.28,0.1,0.895455,1.000000,0.0,-0.652174,0.0,0.333333,-0.333333


In [132]:
T10_y_class = pd.DataFrame(T10_target, columns = ['Y_Quality'])
T10_idx = T10_y_class.index
T10_y_class = T10_y_class.reset_index()
T10_data = pd.concat([T10_data, T10_y_class], axis = 1)
T10_data = T10_data.set_index('index')
T10_data.drop(['Y_Quality'], axis = 1, inplace = True)
T10_data

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK
index,,,,,,,,,,,,,,,,,,,,,
22,0.0,0.0,1.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.962572,0.66,0.0,0.386364,0.111111,1.0,0.584058,0.0,-0.166667,-1.000000
23,0.0,0.0,1.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-45.815738,-29.52,-32.6,-38.275000,-32.111111,-136.0,-180.608692,-13.2,-0.083333,-1.000000
25,0.0,0.0,0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.022073,0.66,0.1,0.581818,1.000000,1.0,0.739130,0.0,-0.083333,0.666667
29,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.606526,0.30,0.1,0.738636,1.000000,1.0,0.304348,0.0,1.083333,0.666667
37,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.544146,0.16,0.1,0.756818,1.000000,1.0,0.763768,0.1,-0.583333,-0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,0.0,-1.0,-0.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.092131,0.30,0.0,0.634091,1.000000,1.0,1.130435,0.1,0.666667,-1.000000
590,0.0,-1.0,-1.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.241843,0.48,0.1,0.900000,1.111111,0.0,-0.086956,0.0,-0.750000,-0.666667
591,0.0,-1.0,-1.6,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.231286,0.02,0.0,0.909091,1.000000,0.0,0.000000,0.0,-0.750000,-0.666667


In [133]:
#13
from pycaret.regression import *
setup_rgs = setup(data = T10_data, target = T10_target, session_id = 4)
top5 = compare_models(sort='MSE', n_select=5)

,Description,Value
0,Session id,4
1,Target,Y_Quality
2,Target type,Regression
3,Original data shape,"(349, 669)"
4,Transformed data shape,"(349, 669)"
5,Transformed train set shape,"(244, 669)"
6,Transformed test set shape,"(105, 669)"
7,Numeric features,668
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.0030,0.0000,0.0045,-0.0502,0.0029,0.0057,0.0260
catboost,CatBoost Regressor,0.0029,0.0000,0.0042,0.0759,0.0027,0.0056,6.9150
en,Elastic Net,0.0030,0.0000,0.0045,-0.0502,0.0029,0.0057,0.0250
xgboost,Extreme Gradient Boosting,0.0031,0.0000,0.0043,-0.0803,0.0028,0.0059,0.1150
llar,Lasso Least Angle Regression,0.0030,0.0000,0.0045,-0.0502,0.0029,0.0057,0.0240
br,Bayesian Ridge,0.0035,0.0000,0.0047,-0.2238,0.0031,0.0066,0.0290
knn,K Neighbors Regressor,0.0034,0.0000,0.0047,-0.1698,0.0030,0.0064,0.0270
dt,Decision Tree Regressor,0.0041,0.0000,0.0056,-0.9387,0.0037,0.0077,0.0320
rf,Random Forest Regressor,0.0032,0.0000,0.0045,-0.1001,0.0029,0.0060,0.3960
et,Extra Trees Regressor,0.0030,0.0000,0.0041,0.0294,0.0027,0.0056,0.2790


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [134]:
T10_total_models = []

T10_model_rf = create_model('rf', fold = 5)
T10_total_models.append(T10_model_rf)

T10_model_gbr = create_model('gbr', fold = 5)
T10_total_models.append(T10_model_gbr)

T10_model_et = create_model('et', fold = 5)
T10_total_models.append(T10_model_et)

T10_model_catboost = create_model('catboost', fold = 5)
T10_total_models.append(T10_model_catboost)

T10_model_xgb = create_model('xgboost', fold = 5)
T10_total_models.append(T10_model_xgb)

T10_model_ada = create_model('ada', fold = 5)
T10_total_models.append(T10_model_ada)

T10_model_lightgbm = create_model('lightgbm', fold = 5)
T10_total_models.append(T10_model_lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0033,0.0000,0.0046,0.0622,0.0030,0.0062
1,0.0033,0.0000,0.0050,0.0319,0.0033,0.0063
2,0.0029,0.0000,0.0043,-0.2064,0.0028,0.0054
3,0.0028,0.0000,0.0037,-0.0722,0.0024,0.0053
4,0.0038,0.0000,0.0053,0.0244,0.0035,0.0072
Mean,0.0032,0.0000,0.0046,-0.0320,0.0030,0.0061
Std,0.0003,0.0000,0.0005,0.0981,0.0004,0.0007


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0030,0.0000,0.0040,0.2839,0.0026,0.0056
1,0.0030,0.0000,0.0045,0.2097,0.0030,0.0058
2,0.0029,0.0000,0.0043,-0.2180,0.0028,0.0055
3,0.0028,0.0000,0.0039,-0.1696,0.0025,0.0052
4,0.0036,0.0000,0.0054,-0.0196,0.0035,0.0068
Mean,0.0031,0.0000,0.0044,0.0173,0.0029,0.0058
Std,0.0003,0.0000,0.0005,0.1999,0.0003,0.0006


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0030,0.0000,0.0041,0.2433,0.0027,0.0056
1,0.0034,0.0000,0.0050,0.0570,0.0032,0.0064
2,0.0028,0.0000,0.0039,0.0235,0.0025,0.0053
3,0.0027,0.0000,0.0033,0.1681,0.0021,0.0050
4,0.0035,0.0000,0.0048,0.1742,0.0032,0.0067
Mean,0.0031,0.0000,0.0042,0.1332,0.0028,0.0058
Std,0.0003,0.0000,0.0006,0.0810,0.0004,0.0007


...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0047,0.0256,0.0031,0.0059
1,0.0034,0.0000,0.0052,-0.0378,0.0034,0.0065
2,0.0028,0.0000,0.0039,0.0098,0.0025,0.0053
3,0.0025,0.0000,0.0037,-0.0834,0.0024,0.0046
4,0.0038,0.0000,0.0057,-0.1292,0.0037,0.0072
Mean,0.0031,0.0000,0.0046,-0.0430,0.0030,0.0059
Std,0.0005,0.0000,0.0007,0.0576,0.0005,0.0009


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0035,0.0000,0.0049,-0.0537,0.0032,0.0065
1,0.0036,0.0000,0.0052,-0.0492,0.0034,0.0068
2,0.0031,0.0000,0.0042,-0.1794,0.0028,0.0057
3,0.0029,0.0000,0.0040,-0.2454,0.0026,0.0055
4,0.0042,0.0000,0.0057,-0.1301,0.0037,0.0080
Mean,0.0034,0.0000,0.0048,-0.1316,0.0031,0.0065
Std,0.0005,0.0000,0.0006,0.0750,0.0004,0.0009


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [135]:
T10_model_rf = tune_model(T10_model_rf, fold=5, choose_better = True)

T10_model_gbr = tune_model(T10_model_gbr, fold=5, choose_better = True)

T10_model_et = tune_model(T10_model_et, fold=5, choose_better = True)

T10_model_catboost = tune_model(T10_model_catboost, fold=5, choose_better = True)

T10_model_xgb = tune_model(T10_model_xgb, fold=5, choose_better = True)

T10_model_ada = tune_model(T10_model_ada, fold=5, choose_better = True)

T10_model_lightgbm = tune_model(T10_model_lightgbm, fold=5, choose_better = True)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0047,0.0252,0.0031,0.0059
1,0.0032,0.0000,0.0051,0.0141,0.0033,0.0061
2,0.0028,0.0000,0.0038,0.0364,0.0025,0.0052
3,0.0026,0.0000,0.0037,-0.0886,0.0024,0.0048
4,0.0035,0.0000,0.0053,-0.0040,0.0035,0.0066
Mean,0.0030,0.0000,0.0045,-0.0034,0.0030,0.0057
Std,0.0003,0.0000,0.0006,0.0446,0.0004,0.0006


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0034,0.0000,0.0051,-0.1290,0.0033,0.0064
1,0.0032,0.0000,0.0048,0.1015,0.0032,0.0060
2,0.0025,0.0000,0.0035,0.1977,0.0023,0.0046
3,0.0026,0.0000,0.0039,-0.1500,0.0025,0.0049
4,0.0034,0.0000,0.0050,0.1105,0.0033,0.0064
Mean,0.0030,0.0000,0.0045,0.0261,0.0029,0.0057
Std,0.0004,0.0000,0.0007,0.1395,0.0004,0.0008


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0031,0.0000,0.0048,-0.0000,0.0031,0.0058
1,0.0032,0.0000,0.0051,-0.0121,0.0034,0.0062
2,0.0028,0.0000,0.0039,-0.0000,0.0025,0.0054
3,0.0026,0.0000,0.0038,-0.0903,0.0024,0.0048
4,0.0034,0.0000,0.0054,-0.0105,0.0035,0.0065
Mean,0.0030,0.0000,0.0046,-0.0226,0.0030,0.0057
Std,0.0003,0.0000,0.0006,0.0342,0.0004,0.0006


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [136]:
T10_tuned_models = [T10_model_rf, T10_model_gbr, T10_model_et, T10_model_catboost, T10_model_xgb, T10_model_ada, T10_model_lightgbm]
T10_tuned_models

[RandomForestRegressor(criterion='absolute_error', max_depth=9,
                       max_features='sqrt', min_impurity_decrease=0.0001,
                       min_samples_leaf=4, min_samples_split=10, n_estimators=30,
                       n_jobs=-1, random_state=4),
 GradientBoostingRegressor(learning_rate=0.15, max_depth=1, max_features='log2',
                           min_impurity_decrease=0.0001, min_samples_leaf=3,
                           min_samples_split=5, n_estimators=190, random_state=4,
                           subsample=0.65),
 ExtraTreesRegressor(n_jobs=-1, random_state=4),
 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin

In [137]:
for model in T10_tuned_models:
    display(predict_model(model))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0034,0.0000,0.0049,0.0171,0.0032,0.0064


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.0,0.531818,0.111111,1.0,0.434783,0.1,-0.666667,0.666667,0.525525,0.530568
245,0.0,0.0,0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.500000,-0.666667,0.531676,0.530506
246,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.250000,-0.333333,0.529952,0.530588
247,0.0,-1.0,-0.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.1,0.972727,1.000000,1.0,0.449275,0.0,0.500000,-1.000000,0.531376,0.530616
248,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,-0.416667,0.666667,0.528473,0.530740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,0.0,0.0,0.0,1.333333,0.0,1.0,...,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.166667,-0.333333,0.526997,0.530571
345,0.0,0.0,-1.2,0.0,0.0,0.0,0.0,2.000000,0.0,1.0,...,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.916667,0.333333,0.537283,0.530430
346,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,1.500000,0.0,1.0,...,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,-0.750000,0.000000,0.527637,0.530359
347,0.0,-1.0,1.4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.750000,0.333333,0.525897,0.530586


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0033,0.0000,0.0048,0.0934,0.0031,0.0063


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.0,0.531818,0.111111,1.0,0.434783,0.1,-0.666667,0.666667,0.525525,0.530827
245,0.0,0.0,0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.500000,-0.666667,0.531676,0.532487
246,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.250000,-0.333333,0.529952,0.530931
247,0.0,-1.0,-0.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.1,0.972727,1.000000,1.0,0.449275,0.0,0.500000,-1.000000,0.531376,0.531045
248,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,-0.416667,0.666667,0.528473,0.528753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,0.0,0.0,0.0,1.333333,0.0,1.0,...,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.166667,-0.333333,0.526997,0.530212
345,0.0,0.0,-1.2,0.0,0.0,0.0,0.0,2.000000,0.0,1.0,...,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.916667,0.333333,0.537283,0.531840
346,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,1.500000,0.0,1.0,...,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,-0.750000,0.000000,0.527637,0.530094
347,0.0,-1.0,1.4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.750000,0.333333,0.525897,0.529410


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0029,0.0000,0.0038,0.4136,0.0025,0.0055


...

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.0,0.531818,0.111111,1.0,0.434783,0.1,-0.666667,0.666667,0.525525,0.529878
245,0.0,0.0,0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.500000,-0.666667,0.531676,0.531130
246,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.250000,-0.333333,0.529952,0.527844
247,0.0,-1.0,-0.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.1,0.972727,1.000000,1.0,0.449275,0.0,0.500000,-1.000000,0.531376,0.531153
248,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,-0.416667,0.666667,0.528473,0.527475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,0.0,0.0,0.0,1.333333,0.0,1.0,...,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.166667,-0.333333,0.526997,0.532280
345,0.0,0.0,-1.2,0.0,0.0,0.0,0.0,2.000000,0.0,1.0,...,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.916667,0.333333,0.537283,0.535911
346,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,1.500000,0.0,1.0,...,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,-0.750000,0.000000,0.527637,0.529630
347,0.0,-1.0,1.4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.750000,0.333333,0.525897,0.527339


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,0.0032,0.0000,0.0045,0.1737,0.0030,0.0060


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.0,0.531818,0.111111,1.0,0.434783,0.1,-0.666667,0.666667,0.525525,0.530360
245,0.0,0.0,0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.500000,-0.666667,0.531676,0.530387
246,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.250000,-0.333333,0.529952,0.530354
247,0.0,-1.0,-0.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.1,0.972727,1.000000,1.0,0.449275,0.0,0.500000,-1.000000,0.531376,0.530529
248,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,-0.416667,0.666667,0.528473,0.530293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,0.0,0.0,0.0,1.333333,0.0,1.0,...,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.166667,-0.333333,0.526997,0.530387
345,0.0,0.0,-1.2,0.0,0.0,0.0,0.0,2.000000,0.0,1.0,...,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.916667,0.333333,0.537283,0.530664
346,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,1.500000,0.0,1.0,...,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,-0.750000,0.000000,0.527637,0.530387
347,0.0,-1.0,1.4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.750000,0.333333,0.525897,0.530463


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.0034,0.0000,0.0050,-0.0000,0.0033,0.0063


,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK,Y_Quality,prediction_label
244,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.0,0.531818,0.111111,1.0,0.434783,0.1,-0.666667,0.666667,0.525525,0.530323
245,0.0,0.0,0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,0.1,0.661364,1.000000,-1.0,-0.608696,0.0,-0.500000,-0.666667,0.531676,0.530323
246,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.0,0.181818,0.000000,0.0,-0.173913,0.0,-0.250000,-0.333333,0.529952,0.530323
247,0.0,-1.0,-0.2,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,0.1,0.972727,1.000000,1.0,0.449275,0.0,0.500000,-1.000000,0.531376,0.530323
248,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-0.9,-0.418182,0.111111,0.0,0.000000,0.0,-0.416667,0.666667,0.528473,0.530323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.0,0.0,-0.4,0.0,0.0,0.0,0.0,1.333333,0.0,1.0,...,0.1,0.568182,0.888889,0.0,-0.521739,0.0,-0.166667,-0.333333,0.526997,0.530323
345,0.0,0.0,-1.2,0.0,0.0,0.0,0.0,2.000000,0.0,1.0,...,0.1,0.184091,0.111111,0.0,-0.224638,0.0,0.916667,0.333333,0.537283,0.530323
346,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,1.500000,0.0,1.0,...,-0.1,-0.150000,0.111111,-1.0,-1.213043,-0.1,-0.750000,0.000000,0.527637,0.530323
347,0.0,-1.0,1.4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,-0.8,-0.288636,0.111111,0.0,-0.086956,0.0,0.750000,0.333333,0.525897,0.530323


In [138]:
T10_final_regression_model = finalize_model(T10_model_et)

In [141]:
T10_prediction = predict_model(T10_final_regression_model, data=T10_data)

In [142]:
T10_datas = T10_prediction['prediction_label']
T10_datas = pd.DataFrame(T10_datas.values, columns = ['Y_Quality'])
T10_datas

,Y_Quality
0,0.517719
1,0.519090
2,0.529362
3,0.532405
4,0.533110
...,...
344,0.529510
345,0.529948
346,0.529308
347,0.526546


In [143]:
T10_xtrain, T10_xtest, T10_ytrain, T10_ytest = train_test_split(T10_datas, T10_class, test_size = 0.1, random_state = 4)

In [145]:
scores = []
for thisC in [*range(1,100)]:
    svc=SVC(kernel='rbf',C=thisC, decision_function_shape = 'ovo')
    model=svc.fit(T10_xtrain,T10_ytrain)
    scoreTrain=model.score(T10_xtrain,T10_ytrain)
    scoreTest=model.score(T10_xtest,T10_ytest)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.990446, test score:1.000000
선형 SVM : C:2, training score:0.990446, test score:1.000000
선형 SVM : C:3, training score:0.990446, test score:1.000000
선형 SVM : C:4, training score:0.990446, test score:1.000000
선형 SVM : C:5, training score:0.990446, test score:1.000000
선형 SVM : C:6, training score:0.990446, test score:1.000000
선형 SVM : C:7, training score:0.990446, test score:1.000000
선형 SVM : C:8, training score:0.990446, test score:1.000000
선형 SVM : C:9, training score:0.990446, test score:1.000000
선형 SVM : C:10, training score:0.993631, test score:1.000000
선형 SVM : C:11, training score:0.993631, test score:1.000000
선형 SVM : C:12, training score:1.000000, test score:1.000000
선형 SVM : C:13, training score:1.000000, test score:1.000000
선형 SVM : C:14, training score:1.000000, test score:1.000000
선형 SVM : C:15, training score:1.000000, test score:1.000000
선형 SVM : C:16, training score:1.000000, test score:1.000000
선형 SVM : C:17, training score:1.000000, test scor

In [146]:
T10_svm = svm.SVC(kernel = 'rbf', decision_function_shape = 'ovo', C = 10)
T10_svm.fit(T10_xtrain, T10_ytrain)
T10_svm.predict(T10_xtest)

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1], dtype=int64)

# Test

In [147]:
test_data = pd.read_csv("./test.csv")
test_data

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,TEST_305,2022-11-05 11:18,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,TEST_306,2022-11-05 16:39,T100304,T_31,2.0,96.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,TEST_307,2022-11-05 16:47,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,TEST_308,2022-11-05 20:53,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
test_data['YEAR'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.year
test_data['MONTH'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.month
test_data['DAY'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.day
test_data['HOUR'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.hour
test_data['MINUTE'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.minute
test_data['DAY_OF_WEEK'] =  pd.to_datetime(test_data['TIMESTAMP']).dt.day_name()

In [149]:
test_data = test_data.drop(['MONTH', 'DAY', 'YEAR', 'MINUTE'], axis = 1)

In [150]:
test_data.drop(colls, inplace = True, axis = 1)

In [151]:
test_data.loc[test_data.DAY_OF_WEEK == 'Monday','DAY_OF_WEEK'] = 1
test_data.loc[test_data.DAY_OF_WEEK == 'Tuesday','DAY_OF_WEEK'] = 2
test_data.loc[test_data.DAY_OF_WEEK == 'Wednesday','DAY_OF_WEEK'] = 3
test_data.loc[test_data.DAY_OF_WEEK == 'Thursday','DAY_OF_WEEK'] = 4
test_data.loc[test_data.DAY_OF_WEEK == 'Friday','DAY_OF_WEEK'] = 5
test_data.loc[test_data.DAY_OF_WEEK == 'Saturday','DAY_OF_WEEK'] = 6
test_data.loc[test_data.DAY_OF_WEEK == 'Sunday','DAY_OF_WEEK'] = 7

test_data.loc[test_data.PRODUCT_CODE == 'A_31','PRODUCT_CODE'] = 1
test_data.loc[test_data.PRODUCT_CODE == 'T_31','PRODUCT_CODE'] = 2
test_data.loc[test_data.PRODUCT_CODE == 'O_31','PRODUCT_CODE'] = 3

test_data.drop(['PRODUCT_ID', 'TIMESTAMP'], axis = 1, inplace = True)
test_data

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK
0,T100306,2,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5
1,T100304,2,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5
2,T100304,2,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,5
3,T010305,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,444.192308,423.0,NaN,NaN,NaN,NaN,NaN,NaN,10,5
4,T010306,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,443.211539,423.0,NaN,NaN,NaN,NaN,NaN,NaN,11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,T100306,2,2.0,91.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,6
306,T100304,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,6
307,T100306,2,2.0,91.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,6
308,T100306,2,2.0,95.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,6


In [152]:
TT050304 = test_data[test_data['LINE'] == 'T050304']
TT050307 = test_data[test_data['LINE'] == 'T050307']
TT010305 = test_data[test_data['LINE'] == 'T010305']
TT010306 = test_data[test_data['LINE'] == 'T010306']
TT100304 = test_data[test_data['LINE'] == 'T100304']
TT100306 = test_data[test_data['LINE'] == 'T100306']
TT050304

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK
7,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,369.462963,354.0,57.74,52.51,54.45,57.99,63.16,1.0,12,6
8,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,370.259259,353.0,53.18,56.02,41.79,47.94,60.72,1.0,12,6
14,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,369.377359,353.0,49.77,54.93,49.85,48.62,63.18,1.0,6,6
35,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,369.660377,352.0,55.22,58.14,48.93,56.16,64.25,1.0,3,7
41,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,368.018519,352.0,49.67,45.00,49.81,55.19,67.17,1.0,23,1
42,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,368.452830,353.0,51.30,47.66,60.00,53.75,62.58,1.0,23,1
52,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,368.320755,353.0,54.06,51.18,52.17,57.75,63.94,1.0,17,3
53,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,361.226415,342.0,54.04,51.25,54.98,57.22,63.33,1.0,17,3
54,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,367.849057,352.0,55.72,57.57,42.08,55.19,66.06,1.0,18,3
61,T050304,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,370.000000,354.0,NaN,NaN,NaN,NaN,NaN,1.0,21,7


In [153]:
TT050304.drop(['LINE'], axis = 1, inplace = True)
TT050307.drop(['LINE'], axis = 1, inplace = True)
TT010305.drop(['LINE'], axis = 1, inplace = True)
TT010306.drop(['LINE'], axis = 1, inplace = True)
TT100304.drop(['LINE'], axis = 1, inplace = True)
TT100306.drop(['LINE'], axis = 1, inplace = True)

In [154]:
TT050304.drop(T050304_col, inplace = True, axis = 1)
TT050307.drop(T050307_col, inplace = True, axis = 1)
TT010305.drop(T010305_col, inplace = True, axis = 1)
TT010306.drop(T010306_col, inplace = True, axis = 1)
TT100304.drop(T100304_col, inplace = True, axis = 1)
TT100306.drop(T100306_col, inplace = True, axis = 1)
TT050304

,PRODUCT_CODE,X_128,X_129,X_132,X_133,X_134,X_135,X_136,X_137,X_138,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK
7,1,18031.0,18031.0,0.19,0.20,0.19,0.2,354.0,354.0,354.0,...,369.462963,354.0,57.74,52.51,54.45,57.99,63.16,1.0,12,6
8,1,18064.0,18064.0,0.19,0.20,0.19,0.2,355.0,354.0,355.0,...,370.259259,353.0,53.18,56.02,41.79,47.94,60.72,1.0,12,6
14,1,295.0,295.0,0.19,0.21,0.19,0.2,60.0,60.0,60.0,...,369.377359,353.0,49.77,54.93,49.85,48.62,63.18,1.0,6,6
35,1,8632.0,8632.0,0.20,0.21,0.20,0.2,249.0,249.0,249.0,...,369.660377,352.0,55.22,58.14,48.93,56.16,64.25,1.0,3,7
41,1,10712.0,10712.0,0.20,0.21,0.20,0.2,293.0,293.0,293.0,...,368.018519,352.0,49.67,45.00,49.81,55.19,67.17,1.0,23,1
42,1,10729.0,10729.0,0.20,0.21,0.19,0.2,293.0,293.0,293.0,...,368.452830,353.0,51.30,47.66,60.00,53.75,62.58,1.0,23,1
52,1,12261.0,12261.0,0.19,0.21,0.19,0.2,330.0,330.0,330.0,...,368.320755,353.0,54.06,51.18,52.17,57.75,63.94,1.0,17,3
53,1,12283.0,12283.0,0.20,0.21,0.20,0.2,330.0,330.0,330.0,...,361.226415,342.0,54.04,51.25,54.98,57.22,63.33,1.0,17,3
54,1,12305.0,12305.0,0.19,0.21,0.19,0.2,330.0,330.0,330.0,...,367.849057,352.0,55.72,57.57,42.08,55.19,66.06,1.0,18,3
61,1,17990.0,17990.0,0.20,0.21,0.20,0.2,436.0,436.0,436.0,...,370.000000,354.0,NaN,NaN,NaN,NaN,NaN,1.0,21,7


In [155]:
TT05 = pd.concat([TT050304, TT050307])
TT01 = pd.concat([TT010305, TT010306])
TT10 = pd.concat([TT100304, TT100306])

In [156]:
TT05.drop(T5drop, inplace = True, axis = 1)
TT01.drop(T0drop, inplace = True, axis = 1)
TT10.drop(T1drop, inplace = True, axis = 1)

In [157]:
TT05 = TT05.fillna(0)
TT01 = TT01.fillna(0)
TT10 = TT10.fillna(0)

In [158]:
TT05_idx = TT05['PRODUCT_CODE'].index
TT05_idx = pd.DataFrame(TT05_idx, columns = ['index'])
TT05_idx

,index
0,7
1,8
2,14
3,35
4,41
5,42
6,52
7,53
8,54
9,61


In [159]:
TT05c = TT05.columns
TT05scaled = T_05scaler.transform(TT05)
TT05scaled
TT05 = pd.DataFrame(TT05scaled, columns = TT05c)
TT05

,PRODUCT_CODE,X_132,X_133,X_134,X_135,X_136,X_137,X_138,X_139,X_140,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,HOUR,DAY_OF_WEEK
0,0.0,-1.0,-1.0,-1.0,0.0,0.055781,0.053753,0.053999,0.449036,0.428571,...,0.739720,2.0,0.318280,-0.115314,0.320714,0.121103,-0.363861,0.0,0.555556,0.4
1,0.0,-1.0,-1.0,-1.0,0.0,0.057809,0.053753,0.056037,0.449036,0.428571,...,1.204775,1.0,0.036189,0.067832,-0.583571,-0.333032,-0.967822,0.0,0.555556,0.4
2,0.0,-1.0,0.0,-1.0,0.0,-0.540568,-0.542596,-0.545084,-1.374656,-1.195011,...,0.689726,1.0,-0.174760,0.010957,-0.007857,-0.302305,-0.358911,0.0,-0.111111,0.4
3,0.0,0.0,0.0,0.0,0.0,-0.157201,-0.159229,-0.159959,-0.013774,-0.052154,...,0.855015,0.0,0.162388,0.178450,-0.073571,0.038409,-0.094059,0.0,-0.444444,0.6
4,0.0,0.0,0.0,0.0,0.0,-0.067951,-0.069980,-0.070301,-0.134986,-0.142857,...,-0.103867,0.0,-0.180946,-0.507175,-0.010714,-0.005423,0.628713,0.0,1.777778,-0.6
5,0.0,0.0,0.0,-1.0,0.0,-0.067951,-0.069980,-0.070301,-0.129477,-0.138322,...,0.149781,1.0,-0.080111,-0.368380,0.717143,-0.070493,-0.507426,0.0,1.777778,-0.6
6,0.0,-1.0,0.0,-1.0,0.0,0.007099,0.005071,0.005094,-0.024793,-0.015873,...,0.072646,1.0,0.090628,-0.184712,0.157857,0.110258,-0.170792,0.0,1.111111,-0.2
7,0.0,0.0,0.0,0.0,0.0,0.007099,0.005071,0.005094,0.002755,-0.011338,...,-4.070605,-10.0,0.089391,-0.181059,0.358571,0.086308,-0.321782,0.0,1.111111,-0.2
8,0.0,-1.0,0.0,-1.0,0.0,0.007099,0.005071,0.005094,-0.019284,-0.020408,...,-0.202836,0.0,0.193319,0.148709,-0.562857,-0.005423,0.353960,0.0,1.222222,-0.2
9,0.0,0.0,0.0,0.0,0.0,0.222110,0.220081,0.221090,0.482094,0.437642,...,1.053362,2.0,-3.253634,-2.855205,-3.568571,-2.499322,-15.997525,0.0,1.555556,0.6


In [160]:
TT05 = pd.concat([TT05_idx, TT05], axis = 1)
TT05 = TT05.set_index('index')

In [161]:
TT01_idx = TT01['PRODUCT_CODE'].index
TT01_idx = pd.DataFrame(TT01_idx, columns = ['index'])
TT01_idx

,index
0,3
1,6
2,10
3,11
4,38
5,40
6,71
7,86
8,88
9,89


In [162]:
TT01c = TT01.columns
TT01scaled = T_01scaler.transform(TT01)
TT01 = pd.DataFrame(TT01scaled, columns = TT01c)
TT01

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK
0,0.0,0.0,0.75,-0.054348,0.0,0.0,0.415730,0.0,-1.0,-1.0,...,-0.585118,-1.336788,1.159091,1.715374,1.636986,0.2,0.215753,0.0,-0.333333,0.000000
1,0.0,0.0,0.75,-0.054348,0.0,0.0,0.449438,0.0,-1.0,-1.0,...,-1.705924,-2.854922,1.424242,1.957184,1.041096,-0.1,0.317564,0.0,0.666667,0.000000
2,0.0,0.0,0.00,-0.054348,0.0,0.0,0.516854,0.0,-1.0,-1.0,...,-0.951125,-0.829016,1.204545,1.582338,0.993151,0.1,0.206885,0.0,0.111111,0.333333
3,0.0,0.0,1.25,-0.054348,0.0,0.0,0.595506,0.0,-1.0,-1.0,...,-1.343554,-1.497409,1.090909,1.374600,1.109589,0.1,0.220187,-0.5,0.000000,0.666667
4,0.0,1.0,-0.50,-0.054348,0.0,0.0,1.505618,0.0,-1.0,-1.0,...,-1.920884,-2.207254,0.954545,1.398276,1.383562,-0.3,0.650353,-0.5,-0.555556,0.666667
5,0.0,1.0,0.75,-0.054348,0.0,0.0,1.584270,0.0,-1.0,-1.0,...,-1.380046,-1.658031,1.628788,2.220402,1.767123,0.1,0.957042,4.5,0.666667,-1.333333
6,0.0,1.0,-0.25,-0.054348,0.0,0.0,0.067416,0.0,-1.0,-1.0,...,-0.420824,-0.549223,1.492424,2.167816,1.863014,0.0,0.284937,0.0,1.000000,0.333333
7,0.0,0.0,-0.25,-0.054348,0.0,0.0,0.258427,0.0,-1.0,-1.0,...,0.051022,-0.720207,1.492424,2.143966,1.369863,0.1,0.711256,5.0,0.111111,-0.666667
8,0.0,16.0,0.00,-0.054348,0.0,0.0,0.258427,0.0,-1.0,-1.0,...,5.506263,3.642487,3.325758,3.852278,3.178082,0.1,0.018446,0.0,0.444444,-0.666667
9,0.0,1.0,-0.25,-0.054348,0.0,0.0,0.258427,0.0,-1.0,-1.0,...,0.291414,0.010363,1.446970,2.243647,1.671233,0.0,0.390890,1.5,0.444444,-0.666667


In [163]:
TT01 = pd.concat([TT01_idx, TT01], axis = 1)
TT01 = TT01.set_index('index')
TT01

,PRODUCT_CODE,X_246,X_247,X_248,X_249,X_250,X_251,X_252,X_253,X_254,...,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,HOUR,DAY_OF_WEEK
index,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.75,-0.054348,0.0,0.0,0.415730,0.0,-1.0,-1.0,...,-0.585118,-1.336788,1.159091,1.715374,1.636986,0.2,0.215753,0.0,-0.333333,0.000000
6,0.0,0.0,0.75,-0.054348,0.0,0.0,0.449438,0.0,-1.0,-1.0,...,-1.705924,-2.854922,1.424242,1.957184,1.041096,-0.1,0.317564,0.0,0.666667,0.000000
10,0.0,0.0,0.00,-0.054348,0.0,0.0,0.516854,0.0,-1.0,-1.0,...,-0.951125,-0.829016,1.204545,1.582338,0.993151,0.1,0.206885,0.0,0.111111,0.333333
11,0.0,0.0,1.25,-0.054348,0.0,0.0,0.595506,0.0,-1.0,-1.0,...,-1.343554,-1.497409,1.090909,1.374600,1.109589,0.1,0.220187,-0.5,0.000000,0.666667
38,0.0,1.0,-0.50,-0.054348,0.0,0.0,1.505618,0.0,-1.0,-1.0,...,-1.920884,-2.207254,0.954545,1.398276,1.383562,-0.3,0.650353,-0.5,-0.555556,0.666667
40,0.0,1.0,0.75,-0.054348,0.0,0.0,1.584270,0.0,-1.0,-1.0,...,-1.380046,-1.658031,1.628788,2.220402,1.767123,0.1,0.957042,4.5,0.666667,-1.333333
71,0.0,1.0,-0.25,-0.054348,0.0,0.0,0.067416,0.0,-1.0,-1.0,...,-0.420824,-0.549223,1.492424,2.167816,1.863014,0.0,0.284937,0.0,1.000000,0.333333
86,0.0,0.0,-0.25,-0.054348,0.0,0.0,0.258427,0.0,-1.0,-1.0,...,0.051022,-0.720207,1.492424,2.143966,1.369863,0.1,0.711256,5.0,0.111111,-0.666667
88,0.0,16.0,0.00,-0.054348,0.0,0.0,0.258427,0.0,-1.0,-1.0,...,5.506263,3.642487,3.325758,3.852278,3.178082,0.1,0.018446,0.0,0.444444,-0.666667


In [164]:
TT10_idx = TT10['PRODUCT_CODE'].index
TT10_idx = pd.DataFrame(TT10_idx, columns = ['index'])
TT10_idx

,index
0,1
1,2
2,15
3,17
4,20
...,...
238,301
239,305
240,307
241,308


In [165]:
TT10c = TT10.columns
TT10scaled = T_10scaler.transform(TT10)
TT10 = pd.DataFrame(TT10scaled, columns = TT10c)
TT10

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK
0,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-1.732246,-1.24,0.1,0.800000,1.000000,0.0,0.086956,0.0,-0.666667,0.000000
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,1.473129,1.90,0.0,0.015909,0.000000,0.0,-0.391304,0.0,-0.166667,0.000000
2,0.0,0.0,1.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,2.522073,1.00,-0.9,-0.343182,0.000000,0.0,-0.173913,0.0,-0.416667,-1.000000
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,3.619002,1.00,0.0,0.490909,0.000000,1.0,0.043478,0.0,1.083333,-0.333333
4,0.0,0.0,-0.2,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,3.346449,2.30,-0.9,-0.293182,0.000000,0.0,0.044927,0.0,-0.750000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,1.000000,0.0,1.0,...,-0.857965,0.18,0.1,0.818182,1.000000,0.0,-0.260870,0.0,0.416667,0.000000
239,0.0,0.0,-0.8,0.0,0.0,0.0,0.0,1.000000,0.0,1.0,...,-0.520154,-0.94,0.0,0.859091,0.555556,0.0,-0.782609,0.0,0.083333,0.333333
240,0.0,0.0,-0.8,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,-0.919386,-0.52,-0.3,-0.222727,0.000000,0.0,-0.521739,0.0,0.500000,0.333333
241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.0,...,-0.275688,-1.14,0.0,0.102273,0.111111,0.0,-0.869565,0.0,0.833333,0.333333


In [166]:
TT10 = pd.concat([TT10_idx, TT10], axis = 1)
TT10 = TT10.set_index('index')
TT10

,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_926,X_927,X_928,X_929,X_930,X_931,X_932,X_933,HOUR,DAY_OF_WEEK
index,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,-0.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,-1.732246,-1.24,0.1,0.800000,1.000000,0.0,0.086956,0.0,-0.666667,0.000000
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,1.473129,1.90,0.0,0.015909,0.000000,0.0,-0.391304,0.0,-0.166667,0.000000
15,0.0,0.0,1.4,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,2.522073,1.00,-0.9,-0.343182,0.000000,0.0,-0.173913,0.0,-0.416667,-1.000000
17,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,3.619002,1.00,0.0,0.490909,0.000000,1.0,0.043478,0.0,1.083333,-0.333333
20,0.0,0.0,-0.2,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,...,3.346449,2.30,-0.9,-0.293182,0.000000,0.0,0.044927,0.0,-0.750000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,1.000000,0.0,1.0,...,-0.857965,0.18,0.1,0.818182,1.000000,0.0,-0.260870,0.0,0.416667,0.000000
305,0.0,0.0,-0.8,0.0,0.0,0.0,0.0,1.000000,0.0,1.0,...,-0.520154,-0.94,0.0,0.859091,0.555556,0.0,-0.782609,0.0,0.083333,0.333333
307,0.0,0.0,-0.8,0.0,0.0,0.0,0.0,0.833333,0.0,1.0,...,-0.919386,-0.52,-0.3,-0.222727,0.000000,0.0,-0.521739,0.0,0.500000,0.333333


In [167]:
from pycaret.regression import *
TT05_prediction = predict_model(T05_final_regression_model, data=TT05)
TT01_prediction = predict_model(T01_final_regression_model, data=TT01)
TT10_prediction = predict_model(T10_final_regression_model, data=TT10)

In [168]:
TT05_prediction = pd.DataFrame(TT05_prediction, columns = ['prediction_label'])
TT01_prediction = pd.DataFrame(TT01_prediction, columns = ['prediction_label'])
TT10_prediction = pd.DataFrame(TT10_prediction, columns = ['prediction_label'])
TT05_prediction

,prediction_label
0,0.523671
1,0.523281
2,0.523020
3,0.523065
4,0.525580
5,0.524485
6,0.523547
7,0.524195
8,0.524575
9,0.524562


In [169]:
TT05_y_class = pd.DataFrame(TT05, columns = ['PRODUCT_CODE'])
TT05_idx = TT05_y_class.index
TT05_y_class = TT05_y_class.reset_index()
TT05_prediction = pd.concat([TT05_prediction, TT05_y_class], axis = 1)
TT05_prediction = TT05_prediction.set_index('index')
TT05_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

TT01_y_class = pd.DataFrame(TT01, columns = ['PRODUCT_CODE'])
TT01_idx = TT01_y_class.index
TT01_y_class = TT01_y_class.reset_index()
TT01_prediction = pd.concat([TT01_prediction, TT01_y_class], axis = 1)
TT01_prediction = TT01_prediction.set_index('index')
TT01_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

TT10_y_class = pd.DataFrame(TT10, columns = ['PRODUCT_CODE'])
TT10_idx = TT10_y_class.index
TT10_y_class = TT10_y_class.reset_index()
TT10_prediction = pd.concat([TT10_prediction, TT10_y_class], axis = 1)
TT10_prediction = TT10_prediction.set_index('index')
TT10_prediction.drop(['PRODUCT_CODE'], axis = 1, inplace = True)

In [170]:
TT05_datas = TT05_prediction['prediction_label']
TT05_datas = pd.DataFrame(TT05_datas.values, columns = ['Y_Quality'])

TT05y_idx= TT05_prediction.index
TT05y_idx = pd.DataFrame(TT05y_idx, columns = ['index'])
TT05_datas = pd.concat([TT05y_idx, TT05_datas], axis = 1)
TT05_datas = TT05_datas.set_index('index')
TT05_datas

,Y_Quality
index,
7,0.523671
8,0.523281
14,0.523020
35,0.523065
41,0.525580
42,0.524485
52,0.523547
53,0.524195
54,0.524575


In [171]:
TT01_datas = TT01_prediction['prediction_label']
TT01_datas = pd.DataFrame(TT01_datas.values, columns = ['Y_Quality'])

TT01y_idx= TT01_prediction.index
TT01y_idx = pd.DataFrame(TT01y_idx, columns = ['index'])
TT01_datas = pd.concat([TT01y_idx, TT01_datas], axis = 1)
TT01_datas = TT01_datas.set_index('index')
TT01_datas

,Y_Quality
index,
3,0.524736
6,0.538837
10,0.538657
11,0.540314
38,0.543050
40,0.531179
71,0.517277
86,0.514354
88,0.514869


In [172]:
TT10_datas = TT10_prediction['prediction_label']
TT10_datas = pd.DataFrame(TT10_datas.values, columns = ['Y_Quality'])

TT10y_idx= TT10_prediction.index
TT10y_idx = pd.DataFrame(TT10y_idx, columns = ['index'])
TT10_datas = pd.concat([TT10y_idx, TT10_datas], axis = 1)
TT10_datas = TT10_datas.set_index('index')
TT10_datas

,Y_Quality
index,
1,0.532476
2,0.531699
15,0.524884
17,0.528250
20,0.527254
...,...
301,0.529073
305,0.528975
307,0.529430


In [173]:
TT05_predictions = T05_svm.predict(TT05_datas)
TT01_predictions = T01_svm.predict(TT01_datas)
TT10_predictions = T10_svm.predict(TT10_datas)

In [174]:
TT05_predictions = pd.DataFrame(TT05_predictions, columns = ['prediction_label'])
TT01_predictions = pd.DataFrame(TT01_predictions, columns = ['prediction_label'])
TT10_predictions = pd.DataFrame(TT10_predictions, columns = ['prediction_label'])

In [175]:
TT05_idxf = TT05_datas.index
TT05_p = pd.DataFrame(TT05_idxf, columns = ['index'])
TT05_predictions = pd.concat([TT05_p, TT05_predictions], axis = 1)
TT05_predictions = TT05_predictions.set_index('index')

TT01_idxf = TT01_datas.index
TT01_p = pd.DataFrame(TT01_idxf, columns = ['index'])
TT01_predictions = pd.concat([TT01_p, TT01_predictions], axis = 1)
TT01_predictions = TT01_predictions.set_index('index')

TT10_idxf = TT10_datas.index
TT10_p = pd.DataFrame(TT10_idxf, columns = ['index'])
TT10_predictions = pd.concat([TT10_p, TT10_predictions], axis = 1)
TT10_predictions = TT10_predictions.set_index('index')

In [176]:
final_test = pd.concat([TT05_predictions, TT01_predictions, TT10_predictions])
final_test

,prediction_label
index,
7,0
8,0
14,0
35,0
41,1
...,...
301,1
305,1
307,1


In [177]:
final_test = final_test.sort_index(ascending=True)
final_test['prediction_label']

index
0      1
1      1
2      1
3      0
4      1
      ..
305    1
306    1
307    1
308    1
309    1
Name: prediction_label, Length: 310, dtype: int64

In [178]:
#52
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['Y_Class'] = final_test['prediction_label']
display(sample_submission.head())
sample_submission.to_csv('line3_otherdrop_robust_regpyc_svmclass_fill0_submission.csv',index=False)

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,0
4,TEST_004,1


In [179]:
final_test['prediction_label'].value_counts()

1    252
0     34
2     24
Name: prediction_label, dtype: int64

In [180]:
np.array(final_test['prediction_label'])

array([1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 2, 2, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1,
       0, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0,
       1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,